In [5]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


print(os.cpu_count())

# Optional: Allow TensorFlow to dynamically adjust the number of threads
#tf.config.threading.set_intra_op_parallelism_threads(32)
#tf.config.threading.set_inter_op_parallelism_threads()

print("Intra-op parallelism threads:", tf.config.threading.get_intra_op_parallelism_threads())
print("Inter-op parallelism threads:", tf.config.threading.get_inter_op_parallelism_threads())

8
Intra-op parallelism threads: 0
Inter-op parallelism threads: 0


In [6]:
sequence_length = 100  # Length of the sequence to be fed into the model
num_targets = 4  # Number of targets in your data
future_target = 75  # The maximum future timestep to predict
train_instances_per_vessel=400

In [11]:
train_data = pd.read_csv("../Datasets/ais_train.csv", delimiter="|")
test_data = pd.read_csv("../Datasets/ais_test.csv", delimiter=",")

In [12]:
vessel_data = pd.read_csv("../Datasets/vessels.csv", delimiter="|")
port_data = pd.read_csv("../Datasets/ports.csv", delimiter="|")

In [13]:
train_data["time"] = pd.to_datetime(train_data["time"])
test_data["time"] = pd.to_datetime(test_data["time"])

In [14]:
train_data = train_data.merge(
    vessel_data[["vesselId", "shippingLineId"]], on="vesselId", how="left"
)

In [15]:
port_data_renamed = pd.DataFrame()
port_data_renamed[["portId", "port_latitude", "port_longitude"]] = port_data[
    ["portId", "latitude", "longitude"]
]
train_data = train_data.merge(port_data_renamed, on="portId", how="left")

In [16]:
print(train_data.columns)

Index(['time', 'cog', 'sog', 'rot', 'heading', 'navstat', 'etaRaw', 'latitude',
       'longitude', 'vesselId', 'portId', 'shippingLineId', 'port_latitude',
       'port_longitude'],
      dtype='object')


In [17]:
train_data_preprocessed = train_data
train_data_preprocessed['navstat'] = train_data_preprocessed['navstat'].astype(int)

# Initialize the OneHotEncoder with sparse=False to get a dense array directly
navstat_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder and transform the 'navstat' column
navstat_encoded = navstat_encoder.fit_transform(train_data_preprocessed[['navstat']])

# Create a DataFrame with the encoded features and set appropriate column names
navstat_encoded_df = pd.DataFrame(navstat_encoded, columns=navstat_encoder.get_feature_names_out(['navstat']))

# Concatenate the encoded features with the original DataFrame
#train_data_preprocessed = pd.concat([train_data_preprocessed, navstat_encoded_df], axis=1)

train_data_preprocessed.loc[train_data_preprocessed["cog"] >= 360, "cog"] = np.nan
train_data_preprocessed.loc[train_data_preprocessed["sog"] >= 1023, "sog"] = np.nan
train_data_preprocessed.loc[train_data_preprocessed["rot"] == -128, "rot"] = np.nan
train_data_preprocessed.loc[train_data_preprocessed["heading"] == 511, "heading"] = (
    np.nan
)


pattern = r"^\d{2}-\d{2} \d{2}:\d{2}$"
train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].where(
    train_data_preprocessed["etaRaw"].str.match(pattern, na=False), np.nan
)


train_data_preprocessed = train_data_preprocessed.sort_values("time")

print(train_data_preprocessed.head())


train_data_preprocessed = (
    train_data_preprocessed.groupby("vesselId")
    .apply(lambda group: group.ffill().bfill())
    .reset_index(drop=True)
)


print(train_data_preprocessed.head())

train_data_preprocessed["heading"] = train_data_preprocessed["heading"].fillna(0)

train_data_preprocessed = train_data_preprocessed.dropna().reset_index(drop=True)


# Replace '00-' in etaRaw with the corresponding month and day from the 'time' column
train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("00-", na=False),
    "01" + train_data_preprocessed["etaRaw"].str[2:],
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("-00", na=False),
    train_data_preprocessed["etaRaw"].str[:2]
    + "-01"
    + train_data_preprocessed["etaRaw"].str[5:],
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains(":60", na=False),
    train_data_preprocessed["etaRaw"].str[:9] + "59",
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("60:", na=False),
    train_data_preprocessed["etaRaw"].str[:6] + "01:00",
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("24:", na=False),
    train_data_preprocessed["etaRaw"].str[:6] + "23:59",
)


train_data_preprocessed["etaRaw"] = pd.to_datetime(
    train_data_preprocessed["time"].dt.year.astype(str)
    + "-"
    + train_data_preprocessed["etaRaw"]
    + ":00",
    format="%Y-%m-%d %H:%M:%S",
)


train_data_preprocessed["seconds_to_eta"] = (
    train_data_preprocessed["etaRaw"] - train_data_preprocessed["time"]
).dt.total_seconds()

train_data_preprocessed = train_data_preprocessed.drop(columns=["etaRaw"])

                 time    cog   sog  rot  heading  navstat       etaRaw  \
0 2024-01-01 00:00:25  284.0   0.7  0.0     88.0        0  01-09 23:00   
1 2024-01-01 00:00:36  109.6   0.0 -6.0    347.0        1  12-29 20:00   
2 2024-01-01 00:01:45  111.0  11.0  0.0    112.0        0  01-02 09:00   
3 2024-01-01 00:03:11   96.4   0.0  0.0    142.0        1  12-31 20:00   
4 2024-01-01 00:03:51  214.0  19.7  0.0    215.0        0  01-25 12:00   

   latitude  longitude                  vesselId                    portId  \
0 -34.74370  -57.85130  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f   
1   8.89440  -79.47939  61e9f3d4b937134a3c4bff1f  634c4de270937fc01c3a7689   
2  39.19065  -76.47567  61e9f436b937134a3c4c0131  61d3847bb7b7526e1adf3d19   
3 -34.41189  151.02067  61e9f3b4b937134a3c4bfe77  61d36f770a1807568ff9a126   
4  35.88379   -5.91636  61e9f41bb937134a3c4c0087  634c4de270937fc01c3a74f3   

             shippingLineId  port_latitude  port_longitude  
0  61ec65aea8cafc0e93f0e9

/var/folders/3t/hy3nmqqx6f70nkbvw0n8lbh80000gn/T/ipykernel_10407/1770637378.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.ffill().bfill())


                 time    cog   sog  rot  heading  navstat       etaRaw  \
0 2024-01-12 14:07:47  308.1  17.1 -6.0    316.0        0  01-08 06:00   
1 2024-01-12 14:31:00  307.6  17.3  5.0    313.0        0  01-14 23:30   
2 2024-01-12 14:57:23  306.8  16.9  5.0    312.0        0  01-14 23:30   
3 2024-01-12 15:18:48  307.9  16.9  6.0    313.0        0  01-14 23:30   
4 2024-01-12 15:39:47  307.0  16.3  7.0    313.0        0  01-14 23:30   

   latitude  longitude                  vesselId                    portId  \
0   7.50361   77.58340  61e9f38eb937134a3c4bfd8b  61d376b393c6feb83e5eb50c   
1   7.57302   77.49505  61e9f38eb937134a3c4bfd8b  61d376d893c6feb83e5eb546   
2   7.65043   77.39404  61e9f38eb937134a3c4bfd8b  61d376d893c6feb83e5eb546   
3   7.71275   77.31394  61e9f38eb937134a3c4bfd8b  61d376d893c6feb83e5eb546   
4   7.77191   77.23585  61e9f38eb937134a3c4bfd8b  61d376d893c6feb83e5eb546   

             shippingLineId  port_latitude  port_longitude  
0  61a8e672f9cba188601e84

In [18]:
train_data_engineered = train_data_preprocessed
train_latitude_radians = np.deg2rad(train_data_engineered["latitude"])
train_longitude_radians = np.deg2rad(train_data_engineered["longitude"])
train_cog_radians = np.deg2rad(train_data_engineered["cog"])
train_heading_radians = np.deg2rad(train_data_engineered["heading"])

port_latitude_radians = np.deg2rad(train_data_engineered["port_latitude"])
port_longitude_radians = np.deg2rad(train_data_engineered["port_longitude"])

train_hour = np.deg2rad(train_data_engineered["time"].dt.hour * 360 / 24)
train_day = np.deg2rad(train_data_engineered["time"].dt.day * 360 / 30)
train_month = np.deg2rad(train_data_engineered["time"].dt.month * 360 / 12)


train_latitude_sin = np.sin(train_latitude_radians)
train_latitude_cos = np.cos(train_latitude_radians)
train_longitude_sin = np.sin(train_longitude_radians)
train_longitude_cos = np.cos(train_longitude_radians)

port_latitude_sin = np.sin(port_latitude_radians)
port_latitude_cos = np.cos(port_latitude_radians)
port_longitude_sin = np.sin(port_longitude_radians)
port_longitude_cos = np.cos(port_longitude_radians)

train_cog_sin = np.sin(train_cog_radians)
train_cog_cos = np.cos(train_cog_radians)

train_heading_sin = np.sin(train_heading_radians)
train_heading_cos = np.cos(train_heading_radians)

train_hour_sin = np.sin(train_hour)
train_hour_cos = np.cos(train_hour)

train_day_sin = np.sin(train_day)
train_day_cos = np.cos(train_day)

train_month_sin = np.sin(train_month)
train_month_cos = np.cos(train_month)


train_data_engineered["latitude_sin"] = train_latitude_sin
train_data_engineered["latitude_cos"] = train_latitude_cos
train_data_engineered["longitude_sin"] = train_longitude_sin
train_data_engineered["longitude_cos"] = train_longitude_cos
train_data_engineered["port_latitude_sin"] = port_latitude_sin
train_data_engineered["port_latitude_cos"] = port_latitude_cos
train_data_engineered["port_longitude_sin"] = port_longitude_sin
train_data_engineered["port_longitude_cos"] = port_longitude_cos
train_data_engineered["cog_sin"] = train_cog_sin
train_data_engineered["cog_cos"] = train_cog_cos
train_data_engineered["heading_sin"] = train_heading_sin
train_data_engineered["heading_cos"] = train_heading_cos

train_data_engineered["hour_sin"] = train_hour_sin
train_data_engineered["hour_cos"] = train_hour_cos
train_data_engineered["day_sin"] = train_day_sin
train_data_engineered["day_cos"] = train_day_cos
train_data_engineered["month_sin"] = train_month_sin
train_data_engineered["month_cos"] = train_month_cos


train_data_engineered["cog_sog_sin"] = (
    train_data_engineered["cog_sin"] * train_data_engineered["sog"]
)
train_data_engineered["cog_sog_cos"] = (
    train_data_engineered["cog_cos"] * train_data_engineered["sog"]
)

train_data_engineered = train_data_engineered.drop(
    columns=[
        "latitude",
        "longitude",
        "cog",
        "heading",
        "portId",
        "cog_sin",
        "cog_cos",
        "sog",
        "shippingLineId", 
        "navstat",
        "heading_sin",
        "heading_cos",
        "port_latitude",
        "port_longitude",
        #"port_latitude_sin",
        #"port_latitude_cos",
        #"port_longitude_sin",
        #"port_longitude_cos",
        "rot", #probably not important, removed because I needed only 18 features
    ],
    axis=1,
)
print(train_data_engineered.columns)

seconds_to_eta_scaler = MinMaxScaler()
rot_scaler = MinMaxScaler()
cog_sog_sin_scaler = MinMaxScaler()
cog_sog_cos_scaler = MinMaxScaler()

train_data_engineered["seconds_to_eta"] = seconds_to_eta_scaler.fit_transform(
    train_data_engineered["seconds_to_eta"].values.reshape(-1, 1)
)
#train_data_engineered["rot"] = rot_scaler.fit_transform(
#    train_data_engineered["rot"].values.reshape(-1, 1)
#)
train_data_engineered["cog_sog_sin"] = cog_sog_sin_scaler.fit_transform(
    train_data_engineered["cog_sog_sin"].values.reshape(-1, 1)
)
train_data_engineered["cog_sog_cos"] = cog_sog_cos_scaler.fit_transform(
    train_data_engineered["cog_sog_cos"].values.reshape(-1, 1)
)

print(train_data_engineered.head())

Index(['time', 'vesselId', 'seconds_to_eta', 'latitude_sin', 'latitude_cos',
       'longitude_sin', 'longitude_cos', 'port_latitude_sin',
       'port_latitude_cos', 'port_longitude_sin', 'port_longitude_cos',
       'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'month_sin', 'month_cos',
       'cog_sog_sin', 'cog_sog_cos'],
      dtype='object')
                 time                  vesselId  seconds_to_eta  latitude_sin  \
0 2024-01-12 14:07:47  61e9f38eb937134a3c4bfd8b        0.250161      0.130589   
1 2024-01-12 14:31:00  61e9f38eb937134a3c4bfd8b        0.263753      0.131790   
2 2024-01-12 14:57:23  61e9f38eb937134a3c4bfd8b        0.263716      0.133129   
3 2024-01-12 15:18:48  61e9f38eb937134a3c4bfd8b        0.263686      0.134207   
4 2024-01-12 15:39:47  61e9f38eb937134a3c4bfd8b        0.263656      0.135230   

   latitude_cos  longitude_sin  longitude_cos  port_latitude_sin  \
0      0.991437       0.976610       0.215018           0.229427   
1      0.991278       0.976

In [19]:

num_features=len(train_data_engineered.columns)-2 +1 # remove time and vesselId, add time_diff
print(num_features)

18


In [22]:
print(train_data_engineered.shape)
duplicates = train_data_engineered.duplicated()
print(train_data_engineered[duplicates].shape)

(1522065, 19)
(0, 19)


In [12]:
print(
    train_data_engineered[
        ["latitude_sin", "latitude_cos", "longitude_sin", "longitude_cos"]
    ].describe()
)

       latitude_sin  latitude_cos  longitude_sin  longitude_cos
count  1.522065e+06  1.522065e+06   1.522065e+06   1.522065e+06
mean   5.674148e-01  7.325459e-01   4.657097e-02   5.090908e-01
std    3.533228e-01  1.287637e-01   5.297501e-01   6.767740e-01
min   -7.376648e-01  3.328656e-01  -1.000000e+00  -9.997826e-01
25%    5.666483e-01  6.245345e-01  -9.085523e-02  -9.307284e-02
50%    6.721562e-01  7.403239e-01   7.383488e-02   9.743727e-01
75%    7.809972e-01  8.197004e-01   3.145059e-01   9.969210e-01
max    9.429743e-01  1.000000e+00   9.923086e-01   1.000000e+00


In [13]:
time_diff_scaler = MinMaxScaler()
time_diff_index = 0


def create_sequences(
    data: pd.DataFrame,
    sequence_length: int,
    future_target: int,
    train_instances_per_vessel: int,
    min_sequence_length: int = 10
):
    """
    Creates sequences of a specified length from the input data for each vessel.

    Args:
        data (pd.DataFrame): The input data containing 'vesselId', 'time', and feature columns.
        sequence_length (int): The desired length of each sequence.
        future_target (int): The maximum number of steps ahead to predict.
        train_instances_per_vessel (int): The number of training instances to generate per vessel.
        min_sequence_length (int): The minimum number of data points required to create a sequence.

    Returns:
        Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, int]:
            A tuple containing the input sequences (X), targets (y), vessel IDs, times, and time_diff_index.
    """
    sequences = []
    targets = []
    vessel_ids = []
    times = []
    all_time_diffs = []
    
    feature_columns = [col for col in data.columns if col not in ["vesselId", "time"]]
    feature_columns.append("time_diff")

    print("Feature Columns:", feature_columns)

    # Group data by 'vesselId'
    grouped = data.groupby("vesselId")

    for vessel_id, group in grouped:
        # Sort the group by 'time'
        group = group.sort_values("time").reset_index(drop=True)

        # Skip vessels with insufficient data
        if len(group) < min_sequence_length:
            continue

        # Calculate time differences to the next instance
        group["time_diff"] = (
            (group["time"].diff(-1).dt.total_seconds()/10**6).abs().fillna(0)
        )
        

        # Convert features to numpy array
        feature_array = group[feature_columns].values

        # Pad sequences if necessary
        if len(feature_array) < sequence_length + future_target:
            padding_length = sequence_length + future_target - len(feature_array)
            padding = np.zeros((padding_length, feature_array.shape[1]))
            feature_array = np.vstack([feature_array, padding])

        # Update the group length after padding
        group_length = len(feature_array)

        num_possible_sequences = group_length - sequence_length - future_target + 1

        # Determine the actual number of sequences we can generate
        actual_instances = min(train_instances_per_vessel, num_possible_sequences)
        if actual_instances <= 0:
            continue

        # Calculate step size
        if actual_instances == 1:
            step_size = 0  # Only one sequence possible
        else:
            step_size = num_possible_sequences / (actual_instances - 1)

        for i in range(actual_instances):
            k = int(i * step_size)
            seq_x = feature_array[k : k + sequence_length].copy()

            # Randomly select future target N steps ahead (1 to future_target inclusive)
            N = np.random.randint(10, future_target)
            target_idx = k + sequence_length + N  # Adjust index accordingly

            # Handle the case where target_idx is beyond the length of the data
            if target_idx >= group_length:
                # Cannot get a target at this index, skip this sequence
                continue

            # Get seq_y
            seq_y = feature_array[target_idx,1:5]  # Assuming target variables are included

            # Calculate cumulative time difference from last entry in seq_x to target
            last_seq_time = group["time"].iloc[min(k + sequence_length - 1, len(group)-1)]
            target_time = group["time"].iloc[min(target_idx, len(group)-1)]
            time_diff_cumulative = (target_time - last_seq_time).total_seconds()/10**6

            # Update the last time_diff in seq_x to be time_diff_cumulative
            time_diff_index = feature_columns.index("time_diff")
            seq_x[-1, time_diff_index] = time_diff_cumulative

            all_time_diffs.extend(seq_x[:, time_diff_index])

            # Append sequences and targets
            sequences.append(seq_x)
            targets.append(seq_y)
            vessel_ids.append(vessel_id)
            times.append(last_seq_time)

    # Convert lists to numpy arrays
    X = np.array(sequences).astype(np.float32)
    y = np.array(targets).astype(np.float32)
    vessel_ids = np.array(vessel_ids)
    times = np.array(times)
    
    return X, y, vessel_ids, times, time_diff_index


In [14]:
import os

print(train_data_engineered.columns)
train_data_sequenced_X = []
train_data_sequenced_Y = []

if os.path.exists(
    f"intermediate/train_data_sequenced_X_{sequence_length}_{future_target}.npy"
):
    print("Loading sequenced data from file")
    train_data_sequenced_X = np.load(
        f"intermediate/train_data_sequenced_X_{sequence_length}_{future_target}.npy",
        allow_pickle=True,
    )
    train_data_sequenced_Y = np.load(
        f"intermediate/train_data_sequenced_Y_{sequence_length}_{future_target}.npy",
        allow_pickle=True,
    )
else:
    print("Creating sequenced data")
    train_data_sequenced_X, train_data_sequenced_Y, vessels, times, time_diff_index = create_sequences(
        train_data_engineered,
        sequence_length=sequence_length,
        future_target=future_target,
        train_instances_per_vessel=train_instances_per_vessel,
    )
    np.save(f"intermediate/train_data_sequenced_X_{sequence_length}_{future_target}.npy",train_data_sequenced_X)
    np.save(f"intermediate/train_data_sequenced_Y_{sequence_length}_{future_target}.npy",train_data_sequenced_Y)

# train_data_shifted_df = train_data_shifted_df.drop(columns=["time"], axis=1)

Index(['time', 'vesselId', 'seconds_to_eta', 'latitude_sin', 'latitude_cos',
       'longitude_sin', 'longitude_cos', 'port_latitude_sin',
       'port_latitude_cos', 'port_longitude_sin', 'port_longitude_cos',
       'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'month_sin', 'month_cos',
       'cog_sog_sin', 'cog_sog_cos'],
      dtype='object')
Creating sequenced data
Feature Columns: ['seconds_to_eta', 'latitude_sin', 'latitude_cos', 'longitude_sin', 'longitude_cos', 'port_latitude_sin', 'port_latitude_cos', 'port_longitude_sin', 'port_longitude_cos', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'month_sin', 'month_cos', 'cog_sog_sin', 'cog_sog_cos', 'time_diff']


In [15]:
from typing import Tuple

def append_last_known_data_test(
    test_data: pd.DataFrame,
    known_data: pd.DataFrame
) -> Tuple[np.ndarray, pd.Series]:
    """
    Groups training data by vesselId and propagates all data from the last known location.
    Pads sequences shorter than sequence_length.

    Args:
        test_data (pd.DataFrame): The test data containing 'vesselId' and 'time'.
        known_data (pd.DataFrame): The known data to extract last known positions.

    Returns:
        Tuple[np.ndarray, pd.Series]: A tuple containing the numpy array of sequences and the original times.
    """
    test_data["time"] = pd.to_datetime(test_data["time"])

    # Check if all vesselIds in test_data are present in known_data
    missing_vessels = test_data.loc[
        ~test_data["vesselId"].isin(known_data["vesselId"]), "vesselId"
    ].unique()
    if missing_vessels.size > 0:
        raise ValueError(
            f"The following vesselIds are missing in known_data: {missing_vessels}"
        )

    # Group known_data by 'vesselId', sort by 'time', and take the last sequence_length records
    grouped_data = (
        known_data.sort_values("time")
        .groupby("vesselId")
        .tail(sequence_length)
        .reset_index(drop=True)
    )

    grouped_data["time_diff"] = (
        (grouped_data.groupby("vesselId")["time"]
        .diff(-1)
        .dt.total_seconds()/10**6)
        .abs()
        .fillna(0)
    )

    test_data_numpy = []
    all_time_diffs = []

    # Determine the feature columns (excluding 'time' and 'vesselId')
    feature_columns = [col for col in grouped_data.columns if col not in ["time", "vesselId"]]

    for idx, row in test_data.iterrows():
        vessel_id = row["vesselId"]
        test_time = row["time"]

        # Extract vessel_data for the current vessel_id
        vessel_data = grouped_data[grouped_data["vesselId"] == vessel_id].copy()

        if vessel_data.empty:
            raise ValueError(f"vesselId {vessel_id} not found in known_data.")

        # Get the actual sequence length
        actual_sequence_length = len(vessel_data)

        # Calculate cumulative time difference and scale it
        last_idx = vessel_data.index[-1]
        time_diff_cumulative = (test_time - vessel_data.loc[last_idx, "time"]).total_seconds()/10**6

        # Update 'time_diff' in vessel_data
        vessel_data.loc[last_idx, "time_diff"] = time_diff_cumulative

        # Drop 'time' and 'vesselId' columns from vessel_data
        vessel_data = vessel_data.drop(["time", "vesselId"], axis=1)

        # Handle padding if the sequence is shorter than sequence_length
        if actual_sequence_length < sequence_length:
            # Calculate how many padding steps are needed
            padding_needed = sequence_length - actual_sequence_length

            # Create padding array with zeros (or a specified padding value)
            padding_array = np.zeros((padding_needed, vessel_data.shape[1]))

            # Optionally, you can set specific values for 'time_diff' in the padding
            time_diff_index = vessel_data.columns.get_loc("time_diff")
            padding_array[:, time_diff_index] = 0  # Set 'time_diff' in padding to zero

            # Convert vessel_data to numpy array
            vessel_array = vessel_data.values

            # Concatenate padding to the beginning of the sequence
            vessel_array = np.vstack((padding_array, vessel_array))
        else:
            # Convert vessel_data to numpy array
            vessel_array = vessel_data.values

        # Ensure the final sequence has the correct length
        assert vessel_array.shape[0] == sequence_length, f"Sequence length mismatch for vessel {vessel_id}"

        # Append to the list
        test_data_numpy.append(vessel_array)
        all_time_diffs.extend(vessel_array[:, vessel_data.columns.get_loc("time_diff")])

    original_time = test_data["time"]
    vessels = test_data["vesselId"]

    return np.array(test_data_numpy).astype(np.float32), original_time



In [16]:
print(train_data_engineered.columns)
test_data_X=0

if os.path.exists(
    f"intermediate/test_data_sequenced_X_{sequence_length}_{future_target}.npy"
    ):
    print("loading test data")
    test_data_X=np.load(f"intermediate/test_data_sequenced_X_{sequence_length}_{future_target}.npy", allow_pickle=True)
else:
    print("creating test data")
    test_data_X, test_time_diff_index = append_last_known_data_test(test_data, train_data_engineered)
    np.save(f"intermediate/test_data_sequenced_X_{sequence_length}_{future_target}.npy", test_data_X)

Index(['time', 'vesselId', 'seconds_to_eta', 'latitude_sin', 'latitude_cos',
       'longitude_sin', 'longitude_cos', 'port_latitude_sin',
       'port_latitude_cos', 'port_longitude_sin', 'port_longitude_cos',
       'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'month_sin', 'month_cos',
       'cog_sog_sin', 'cog_sog_cos'],
      dtype='object')
creating test data


In [17]:
print(test_data_X[-2,-1,:])
print(train_data_sequenced_X[9,-1,:])

[ 0.26093897  0.85954654  0.51105744  0.37444007  0.9272511   0.8085266
  0.5884597   0.18856698  0.9820603  -0.9659258   0.25881904  0.9945219
  0.10452846  0.5        -0.8660254   0.4152345   0.43379942  0.448658  ]
[ 0.29949883  0.15565558  0.9878114  -0.98348236  0.18100391  0.56812304
  0.8229436   0.666312   -0.74567306 -0.8660254   0.5        -0.40673664
  0.9135454   0.8660254   0.5         0.51163256  0.46576107  0.398482  ]


In [18]:

last_lat_radians = np.arctan2(test_data_X[0,49,2], test_data_X[0,49,3])
last_long_radians = np.arctan2(test_data_X[0,49,4],test_data_X[0,49,5])

print(train_data_sequenced_X.shape)

last_lat_radians_train = np.arctan2(train_data_sequenced_X[20,49,2], train_data_sequenced_X[20,49,3])
last_long_radians_train = np.arctan2(train_data_sequenced_X[20,49,4],train_data_sequenced_X[20,49,5])

last_lat_radians_val = np.arctan2(train_data_sequenced_Y[0,0], train_data_sequenced_Y[0,1])
last_long_radians_val = np.arctan2(train_data_sequenced_Y[0,2],train_data_sequenced_Y[0,3])

last_lat_degrees=np.rad2deg(last_lat_radians)
last_long_degrees=np.rad2deg(last_long_radians)

last_lat_degrees_train=np.rad2deg(last_lat_radians_train)
last_long_degrees_train=np.rad2deg(last_long_radians_train)

last_lat_degrees_val=np.rad2deg(last_lat_radians_val)
last_long_degrees_val=np.rad2deg(last_long_radians_val)



print(last_lat_degrees,last_long_degrees)
print(last_lat_degrees_val,last_long_degrees_val)


(269350, 100, 18)
139.03401 16.680733
1.2560499 103.865395


In [19]:
print(train_data_sequenced_X[-5:,49,:])

[[ 2.5765526e-01  7.9402786e-01  6.0788137e-01  6.3823074e-02
   9.9796122e-01  7.9242867e-01  6.0996461e-01  8.4170602e-02
   9.9645138e-01 -5.0000000e-01 -8.6602539e-01  7.4314481e-01
   6.6913062e-01  5.0000000e-01 -8.6602539e-01  5.7660651e-01
   5.5071527e-01  1.2960000e-03]
 [ 2.5954130e-01  8.0440664e-01  5.9407908e-01  9.3520992e-02
   9.9561733e-01  8.0225635e-01  5.9697968e-01  1.2530437e-01
   9.9211836e-01 -9.6592581e-01  2.5881904e-01  7.4314481e-01
   6.6913062e-01  5.0000000e-01 -8.6602539e-01  5.6807750e-01
   5.2291608e-01  1.2340000e-03]
 [ 2.5909337e-01  8.0808425e-01  5.8906686e-01  1.3075615e-01
   9.9141455e-01  8.0626911e-01  5.9154892e-01  1.2822442e-01
   9.9174517e-01  2.5881904e-01  9.6592581e-01  8.6602539e-01
   5.0000000e-01  5.0000000e-01 -8.6602539e-01  5.7206762e-01
   5.1797235e-01  1.1960000e-03]
 [ 2.5864321e-01  8.0759597e-01  5.8973622e-01  1.5169229e-01
   9.8842776e-01  8.0794704e-01  5.8925509e-01  1.5877557e-01
   9.8731470e-01  1.0000000e+00  

In [20]:
@tf.keras.utils.register_keras_serializable()
class ContinuousPositionalEncoding(layers.Layer):
    def __init__(self, d_model, **kwargs):
        """
        Initializes the ContinuousPositionalEncoding layer.

        Args:
            d_model (int): The dimensionality of the model.
            **kwargs: Additional keyword arguments for the Layer base class.
        """
        super(ContinuousPositionalEncoding, self).__init__(**kwargs)
        self.d_model = d_model

    @tf.function
    def call(self, time_differences):
        """
        Computes the positional encoding based on cumulative time differences.

        Args:
            time_differences (tf.Tensor): A tensor of shape (batch_size, sequence_length)
                                          containing cumulative time differences.

        Returns:
            tf.Tensor: Positional encoding tensor of shape (batch_size, sequence_length, d_model).
        """
        # Compute cumulative time positions
        time_positions = tf.cumsum(time_differences, axis=1)  # Shape: (batch_size, sequence_length)
        position = tf.expand_dims(time_positions, axis=-1)    # Shape: (batch_size, sequence_length, 1)

        # Compute the angle rates
        i = tf.range(self.d_model, dtype=tf.float32)          # Shape: (d_model,)
        i = tf.reshape(i, (1, 1, self.d_model))               # Shape: (1, 1, d_model)
        angle_rates = 1 / tf.pow(10000.0, (2 * (i // 2)) / tf.cast(self.d_model, tf.float32))  # Shape: (1, 1, d_model)
        angle_rads = position * angle_rates                   # Shape: (batch_size, sequence_length, d_model)

        # Apply sin to even indices and cos to odd indices
        sines = tf.sin(angle_rads[..., 0::2])                 # Shape: (batch_size, sequence_length, d_model/2)
        cosines = tf.cos(angle_rads[..., 1::2])               # Shape: (batch_size, sequence_length, d_model/2)

        # Concatenate sines and cosines along the last axis
        pos_encoding = tf.concat([sines, cosines], axis=-1)   # Shape: (batch_size, sequence_length, d_model)

        return pos_encoding



In [21]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Layer Normalization
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    # Multi-Head Attention
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs  # Residual Connection

    # Feed-Forward Network
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Dense(ff_dim, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(inputs.shape[-1])(x)
    return x + res  # Residual Connection


In [22]:
def build_transformer_model(
    sequence_length,
    num_features,
    num_targets,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0
):
    feature_inputs = keras.Input(shape=(sequence_length, num_features), name='feature_inputs')
    masked_inputs = keras.layers.Masking(mask_value=0.0)(feature_inputs)

    cumulative_time_inputs = keras.Input(shape=(sequence_length,), name='cumulative_time_inputs')

    # Add Positional Encoding using the custom layer
    positional_encoding_layer = ContinuousPositionalEncoding(num_features)
    positional_encoding = positional_encoding_layer(cumulative_time_inputs)
    x = layers.Add()([masked_inputs, positional_encoding])

    # Transformer Blocks
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D()(x)

    # MLP for Regression
    for units in mlp_units:
        x = layers.Dense(units, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)

    outputs = layers.Dense(num_targets)(x)
    model = keras.Model(inputs=[masked_inputs, cumulative_time_inputs], outputs=outputs)
    return model



In [23]:
def get_aggregated_time_diff(data):
    aggregated_time_diff=data[:,:,-1]
    
    # Step 2: Shift time_diff_next by one position to create time_diff_previous
    # Set the first entry in each sequence to 0 to represent no previous time difference
    aggregated_time_diff = np.roll(aggregated_time_diff, shift=1, axis=1)
    aggregated_time_diff[:, 0] = 0  # Set the first entry of each sequence to 0

    # Step 3: Calculate the cumulative sum of time_diff_previous along each sequence
    aggregated_time_diff = np.cumsum(aggregated_time_diff, axis=1)  # Shape (1471269, sequence_length)
    return(aggregated_time_diff)

In [24]:
model = build_transformer_model(
    sequence_length=sequence_length,
    num_features=num_features,
    num_targets=num_targets,  # Define this variable based on your task
    head_size=num_features,
    num_heads=1,
    ff_dim=num_features*4,
    num_transformer_blocks=6,
    mlp_units=[num_features*4],
    dropout=0.1,
    mlp_dropout=0.1,
)


model.compile(
    loss="mean_squared_error",
    optimizer=keras.optimizers.Adam(),
    metrics=["mae"],
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ cumulative_time_in… │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ keras_tensorCLONE   │ (None, 100, 18)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ continuous_positio… │ (None, 100, 18)   │          0 │ cumulative_time_… │
│ (ContinuousPositio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 100, 18)   │          0 │ keras_tensorCLON… │
│                     │                   │            │ continuous_posit… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 100, 18)   │         36 │ add[1][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 100, 18)   │      1,368 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 100, 18)   │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 100, 18)   │          0 │ dropout_1[1][0],  │
│                     │                   │            │ add[1][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 100, 18)   │         36 │ add_1[1][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 100, 72)   │      1,368 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 100, 72)   │          0 │ dense[1][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 100, 18)   │      1,314 │ dropout_2[1][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 100, 18)   │          0 │ dense_1[1][0],    │
│                     │                   │            │ add_1[1][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 100, 18)   │         36 │ add_2[1][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 100, 18)   │      1,368 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 100, 18)   │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 100, 18)   │          0 │ dropout_4[1][0],  │
│                     │                   │            │ add_2[1][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 100, 18)   │         36 │ add_3[1][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 26,392 (103.09 KB)

 Trainable params: 26,392 (103.09 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
from sklearn.model_selection import train_test_split
import gc

train_data_sequenced_X=train_data_sequenced_X.astype(np.float32)
train_data_sequenced_Y=train_data_sequenced_Y.astype(np.float32)


X_train_1, X_val_1, X_train_2, X_val_2, y_train, y_val = train_test_split(
    train_data_sequenced_X,  # First part of input_train
    get_aggregated_time_diff(train_data_sequenced_X),  # todo verify this is correct
    train_data_sequenced_Y,  # Target array
    test_size=0.1,
    random_state=42
)

#del train_data_sequenced_X
#gc.collect() 

# Aggregate the training and testing inputs into lists for model input
X_train = [X_train_1, X_train_2]
X_val = [X_val_1, X_val_2]


In [26]:
def create_dataset(features, time_diffs, targets, batch_size):
    # Create the dataset
    dataset = tf.data.Dataset.from_tensor_slices(((features, time_diffs), targets))
    
    # Shuffle, batch, and prefetch in one pipeline
    dataset = dataset.shuffle(buffer_size=10000) \
                     .batch(batch_size) \
                     .prefetch(tf.data.AUTOTUNE)
    return dataset

# Create training and validation datasets using the optimized function
train_dataset = create_dataset(X_train_1, X_train_2, y_train, batch_size=1024)
val_dataset = create_dataset(X_val_1, X_val_2, y_val, batch_size=1024)


In [ ]:

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Metric to monitor
    patience=5,          # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored metric
)

# Define the ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    filepath='training_checkpoint/{epoch:02d}.keras',  # Filepath to save the model
    save_freq=25,
    save_best_only=False  # Save the model at the specified frequency, not just the best model
)

# Assuming model, train_dataset, and val_dataset are already defined
# Train the model with the EarlyStopping callback
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=[early_stopping,checkpoint]
)

In [ ]:
#model=keras.models.load_model(f"transformer_{sequence_length}_{future_target}.keras")

In [ ]:
# Predict future positions
prediction_x=[test_data_X,get_aggregated_time_diff(test_data_X)]
predictions = model.predict(prediction_x)

predictions=np.array(predictions).astype("float32")
print(predictions.dtype)

lat_predictions_radians = np.arctan2(predictions[:,0], predictions[:,1])
long_predictions_radians = np.arctan2(predictions[:,2],predictions[:,3])

# Convert radians to degrees
lat_predictions_degrees = np.rad2deg(lat_predictions_radians)
long_predictions_degrees = np.rad2deg(long_predictions_radians)



In [ ]:
print(lat_predictions_degrees)
print(long_predictions_degrees)

In [ ]:
predictions = pd.DataFrame({
        'ID': range(len(lat_predictions_degrees)),
        'longitude_predicted': long_predictions_degrees,
        'latitude_predicted': lat_predictions_degrees
    })
if pd.isna(predictions["latitude_predicted"]).any():
    print("oh no")
print(predictions.columns)

In [ ]:
predictions.to_csv("predictions.csv", index=False)

In [ ]:
print(len(predictions))
print(len(test_data))
print(predictions["longitude_predicted"])